In [1]:
clean_up = True
%run StdPackages.ipynb
d['gams'] = os.path.join(d['CGE'],'gams')

The file_gams_py_gdb2.gdx is still active and was not deleted.


## Test A: Production version 1
Investigate how nested CES and normalized CES sectors work. This goes through identical steps for:
* CES1: A simple CES sector.
* CES2: A nested CES sector (2 nests).
* CES1_norm: Normalized CES sector.
* CES2_norm: Nested and normalized CES sector.

### Run baseline:

*Global settings:*

In [2]:
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,3)})

*Init nesting structures:*

In [3]:
data_str = os.path.join(d['data'], 'Nestings.xlsx')
read_trees = {'CES1': {'OneNest': {'f': 'CES'}}, 'CES2': {'TwoNests': {'f':'CES'}},
              'CES1_norm': {'OneNest': {'f':'CES_norm'}}, 'CES2_norm': {'TwoNests': {'f':'CES_norm'}}}
Trees = {k: NestingTree.AggTree_from_data(data_str, read_trees = read_trees[k], name = k)() for k in read_trees} # init trees

*Init production modules:*

In [4]:
ws = gams.GamsWorkspace(working_directory=d['work'])
Ps = {k: CGE_ProductionV1.Production(tree = Trees[k], glob = glob, ns = {}, s_kwargs = {'ws': ws}) for k in Trees}

*Init states:*

In [5]:
[P.compile(initDB=True) for P in Ps.values()];

*Write text:*

In [6]:
[P.write() for P in Ps.values()];

*Run models (with same workspace):*

In [7]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}

### Calibrate:

*Change state:*

In [8]:
[setattr(p.s,'state','C') for p in Ps.values()];

*Write:*

In [9]:
[P.write() for P in Ps.values()];

*Re-run:*

In [10]:
Ms = {k: Ps[k].run(exportTo=d['work'], ws=ws,**{'cns': 'CONOPT4'}) for k in Ps}